In [1]:
from scripts.community_scoring import *
from scripts.graph_connector import query_executor
from scripts.neo4j_to_networkx import graph_from_cypher
from scripts.louvain_community_detection import *
from scripts.sample_queries import *

In [2]:
# query neo4j and read into in-memory graph
data = query_executor(query_all_data)
G = graph_from_cypher(data)

Node or Edge import error - Type error - continuing
Node or Edge import error - Attribute error - continuing
Node or Edge import error - Type error - continuing
Node or Edge import error - Attribute error - continuing
total nodes: 25200, total edges: 36828


In [3]:
count_fraud_indicators(G)

Total potential fraud edges: 3384


3384

In [4]:
# run louvain algo and
# returns a list of sets where each set is a community of nodes
communities = louvain_community(G)

Generating communities...
Community detection elapsed time: 8.53710675239563
total communities with more than one node: 69


In [5]:
# these functions will select the top N largest communities from 
# the communities list. Then will use those selected communities 
# to create a subset of the original graph 
subcommunities = select_communities(communities, n=5)
subgraph = make_subgraph(G, subcommunities)

In [6]:
# assigning colors to each community
colors = community_colors(subcommunities)

begin coloring
done with coloring


In [7]:
# drawing the communitys and writing the image to a flat file
# draw_communities(subgraph, colors, subcommunities)

In [8]:
# writing the communities to a json file
write_communities_json(subcommunities)

So now we have a list of communities. The list contains sets, where each set is a community of individual nodes. 
The next task is to determine how to measure the significance of each community. 
This can be measures in the following ways:
    1: rank by size - count the nodes in the community, the most dense community is the most significant
    2: density of edges within the community - for a community, measure the number of edges. This can be done by 
    calculating the number of edges in the community to the max possible number of edges within that community. 
    Density formula is D = 2E / (N(N-1)) where E is number of edges and N is number of nodes
    3: Calculate the modularity contribution. Modularity is the strength of connectivity with in a community versus outside the community
    
The next unit of work is scoring the communities based on potential fraud indicators
    1: todo - fill this seciton out. 

In [9]:
# need to write some code that can explore the individual communities
# this will lead into finding the density of the communities
sub_largest = select_communities(communities, n=1)
subgraph_large = make_subgraph(G, sub_largest)

nodes = subgraph_large.nodes()
edges = subgraph_large.edges()

print("Nodes:", nodes)
print("Edges:", edges)
# so below we get a list of nodes in the graph and 
# a list of tuples of edges, where in each tuple [0] is the node id
# and [1:] is the edge ids associated with the nodes

Nodes: [15956, 10, 16412, 15957, 24610, 8227, 8228, 8229, 8230, 8232, 8233, 8234, 8235, 8236, 16291, 8238, 19548, 8241, 8242, 8243, 8244, 8245, 16436, 8247, 8248, 8249, 8251, 8252, 8253, 8254, 5538, 16294, 8257, 8259, 8266, 8267, 116, 8329, 9802, 8360, 16561, 16562, 16563, 16564, 16565, 16568, 16569, 16570, 16571, 16572, 16573, 16574, 16575, 16576, 16577, 16578, 16579, 16580, 16581, 16582, 16583, 8391, 16584, 204, 16323, 16599, 8410, 16327, 9815, 8427, 8428, 8429, 8430, 8431, 8432, 8433, 8434, 8435, 8436, 8438, 8439, 8440, 8441, 8442, 8443, 8444, 8445, 8446, 8447, 16333, 8450, 8451, 8452, 8453, 8454, 8455, 8459, 8460, 16658, 16339, 16671, 299, 8525, 8526, 8528, 8529, 8530, 8532, 16350, 8535, 8536, 8537, 8538, 8539, 8540, 8541, 8542, 351, 352, 8545, 354, 8546, 8548, 355, 357, 8550, 8552, 8551, 358, 361, 362, 359, 364, 366, 367, 368, 369, 370, 363, 372, 373, 374, 375, 376, 377, 378, 380, 381, 19619, 16808, 16809, 16810, 16811, 16812, 16813, 16814, 16815, 16816, 16817, 16818, 16819, 16820

In [10]:
print(subgraph_large.nodes(data=True))

[(15956, {'labels': frozenset({'label'}), 'properties': {'constructcode': 'C01', 'scanned': True, 'createdtimestamp': '2024-05-03T16:38:33.000Z', 'ServiceTypeCode': 1, 'detailsequence': 42318670, 'mailclasscode': 'G0', 'spef': False, 'applicationid': 92, 'parcelidcode': '8ed17925c529f449870817428d57b416db1dbdfdcc8b780d1ee67621fbf352cc'}}), (10, {'labels': frozenset({'mailer'}), 'properties': {'MID_LAST_UPD_DATE': '2014-06-10', 'mailerid': 'c7194a75c24dc18ebb95e4a09871dc6c2cff1a18ef000ea29099844931e189b2', 'MID_LAST_UPD_USERID': 'e8f5646ff024e05374c07b7bd0204334a6aacf0e870dcdbd2eb14aed00d0df66'}}), (16412, {'labels': frozenset({'label'}), 'properties': {'constructcode': 'C03', 'scanned': True, 'createdtimestamp': '2024-05-01T16:37:13.000Z', 'ServiceTypeCode': 346, 'detailsequence': 42115134, 'mailclasscode': 'G1', 'spef': False, 'applicationid': 92, 'parcelidcode': 'a2eddd726af06a8af8a675b834c1a077050a3a1562fddb21122a955a937117b8'}}), (15957, {'labels': frozenset({'label'}), 'properties

In [11]:
print(subgraph_large.edges(data=True))

[(15956, 1380, {'type_': 'part_of', 'properties': {'weight': 1}}), (15956, 22325, {'type_': 'attaches_to', 'properties': {'weight': 1}}), (10, 350, {'type_': 'mails', 'properties': {'weight': 1}}), (10, 351, {'type_': 'mails', 'properties': {'weight': 1}}), (10, 352, {'type_': 'mails', 'properties': {'weight': 1}}), (10, 353, {'type_': 'mails', 'properties': {'weight': 1}}), (10, 354, {'type_': 'mails', 'properties': {'weight': 1}}), (10, 355, {'type_': 'mails', 'properties': {'weight': 1}}), (10, 356, {'type_': 'mails', 'properties': {'weight': 1}}), (10, 357, {'type_': 'mails', 'properties': {'weight': 1}}), (10, 358, {'type_': 'mails', 'properties': {'weight': 1}}), (10, 359, {'type_': 'mails', 'properties': {'weight': 1}}), (10, 361, {'type_': 'mails', 'properties': {'weight': 1}}), (10, 362, {'type_': 'mails', 'properties': {'weight': 1}}), (10, 363, {'type_': 'mails', 'properties': {'weight': 1}}), (10, 364, {'type_': 'mails', 'properties': {'weight': 1}}), (10, 366, {'type_': 'm